Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [0]:
import os
data_root = './data'
pickle_file = os.path.join(data_root, 'notMNIST.pickle')
print("{}".format(pickle_file))

In [0]:
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

In [0]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

In [0]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [0]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [0]:
num_steps = 801

with tf.device('/cpu:0'), tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [0]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [0]:
num_steps = 3001

with tf.device('/cpu:0'), tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [0]:
def define_state(self):
  with self.graph.as_default():
    # Input Layer placeholders for data and labels
    self.dataset_ph = tf.placeholder(tf.float32,
                                     shape=(None, self.n_features), name="dataset")
    self.labels_ph = tf.placeholder(tf.float32,
                                    shape=(None, self.label_size), name="labels")
    self.dropout_ph = tf.placeholder(tf.float32,name="dropout")
    # Use truncated normal initializer
    tr_norm_init = tf.truncated_normal_initializer()

    # Hidden NN Layer:
    with tf.variable_scope("NN", initializer=tr_norm_init) as scope:
      nn_weights = tf.get_variable("Weights", [self.n_features, self.hidden_size])
      nn_biases    = tf.get_variable("Biases", [self.hidden_size], initializer=tf.zeros_initializer)

    # Softmax Score Layer:
    with tf.variable_scope("Softmax", initializer=tr_norm_init) as scope:
      sm_weights = tf.get_variable("Weights", [self.hidden_size, self.label_size])
      sm_biases    = tf.get_variable("Biases", [self.label_size], initializer=tf.zeros_initializer)

    return

In [0]:
def define_computation(self):
  with self.graph.as_default():
    # Input/Placeholder to NN
    with tf.variable_scope("NN", reuse=True) as scope:
      nn_weights = tf.get_variable("Weights")
      nn_biases = tf.get_variable("Biases")

    Z1 = tf.matmul(self.dataset_ph, nn_weights) + nn_biases
    A1 = tf.nn.relu(Z1, "ReLU")
    # Dropout at Activation
    Ad1 = tf.nn.dropout(A1, self.dropout_ph)
  
    # NN to SoftmaxScore
    with tf.variable_scope("Softmax", reuse=True) as scope:
      sm_weights = tf.get_variable("Weights")
      sm_biases    = tf.get_variable("Biases")
    Z2 = tf.matmul(Ad1, sm_weights) + sm_biases
    # Dropout at Output
    logits = tf.nn.dropout(Z2, self.dropout_ph)  
    
    # SoftmaxScore to Prediction
    self.prediction_op = tf.nn.softmax(logits, name="Prediction")

    # Cross Entropy Loss
    XEnt = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=self.labels_ph)
  
    # Scalar Loss
    self.loss_op = tf.reduce_mean(XEnt)
  
    # Accuracy
    P = tf.argmax(self.prediction_op, axis=1)
    L = tf.argmax(self.labels_ph, axis=1)
    self.accuracy_op = 100.0 * tf.reduce_mean(tf.to_float(tf.equal(P,L)))
  
    # Optimizer
    self.train_op = tf.train.GradientDescentOptimizer(self.lr).minimize(self.loss_op)


In [0]:
# Randomly picks a batch of training and validation data
def get_data(self, dataset, labels):
  n = len(dataset)
  perm = np.random.randint(n, size=self.batch_size)
  return dataset[perm], labels[perm]

In [0]:
def run_epochs(self):
  best_val_epoch = 0
  best_val_acc      = -float('inf')

  with tf.device('/cpu:0'), tf.Session(graph=self.graph) as s:
    initer = tf.global_variables_initializer()
    s.run(initer)
    print("Initialized")

    for step in range(self.n_epochs):
      # Generate a minibatch.
      batch_data, batch_labels = self.get_data(train_dataset, train_labels)
      # Dictionary: Feeds Training Minibatch.
      feed_dict = {self.dataset_ph: batch_data, self.labels_ph: batch_labels, self.dropout_ph:self.dropout}
      _, tr_loss, tr_acc = s.run( [self.train_op, self.loss_op, self.accuracy_op], feed_dict=feed_dict)

      if ((step != 0) and (step % 500 == 0)):
        # Track validationation accuracy
        feed_dict = {self.dataset_ph: valid_dataset, self.labels_ph: valid_labels, self.dropout_ph:1.0}
        val_acc = s.run(self.accuracy_op, feed_dict=feed_dict)

        print("Minibatch Step=%d"%step)
        print("------------------------------")
        print("Training Loss=%.2f: Accuracy=%.2f%%" % (tr_loss, tr_acc))
        print("Validation Accuracy: %.2f%%" % val_acc)
      
        # Remember the epoch when best validation accuracy was realized
        # Stop early if average validation accuracy is not improving for a few steps
        if val_acc > best_val_acc:
          best_val_epoch = step
          best_val_acc = val_acc
          
        if step >= (best_val_epoch + self.early_stop):
          print("Step %d: Best_Epoch %d: Early_Stop %d" % (step, best_val_epoch, self.early_stop))
          print("Terminate training early. Validation accuracy has flattened at %.2f%%"% best_val_acc)
          break
      
    print("Training Completed: ")
    print("-------------------------------")
    feed_dict = {self.dataset_ph: test_dataset, self.labels_ph: test_labels, self.dropout_ph:1.0}
    test_acc = s.run( self.accuracy_op, feed_dict=feed_dict)
    print("Test Accuracy: %.2f%%" % test_acc)

In [0]:
# Dimensions
# image_size = 28
# num_labels = 10
# hidden layer
class ImageModel():
  dropout       = 0.9
  lr                    = 0.001
  hidden_size = 1024
  label_size    = num_labels
  n_features   = image_size*image_size
  batch_size   = batch_size
  n_epochs     = 100000
  early_stop   = 1000

  define_state = define_state
  define_computation = define_computation  
  get_data = get_data
  run_epochs = run_epochs  

  def __init__(self):
    self.graph = tf.Graph()
    
imMod = ImageModel()    

In [0]:
imMod.define_state()

In [0]:
imMod.define_computation()

In [0]:
imMod.run_epochs()